In [ ]:
!git clone https://github.com/acmarkes/traffic-sign-recog.git
%cd /content/traffic-sign-recog
!pip install gdown

import gdown
imgs = 'https://drive.google.com/uc?id=17an3cKGO1I2WlBPSa6zcAucL-jISSHEn'
labels = 'https://drive.google.com/uc?id=17gFHTQvU0utSNm3XMv2I8Ud4IzuqDLTl'

gdown.download(imgs, 'procTrainImages.joblib', quiet=False)
gdown.download(labels, 'train_labels.joblib', quiet=False)


In [ ]:
import joblib
from matplotlib import pyplot as plt
import numpy as np
from datetime import datetime as dt

import tensorflow as tf
from tensorflow import keras

import utils
from resnet import ResNet, Resnet50

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
procTrainImages = joblib.load('procTrainImages.joblib')
print(procTrainImages.shape)
utils.sample_images(procTrainImages, seed_num=42)
trainLabels = joblib.load('train_labels.joblib')
print(len(trainLabels))


In [ ]:
img_tweaks = {
    'featurewise_center':True,
    'featurewise_std_normalization':True,
    'rotation_range':20,
    'width_shift_range':0.2,
    'height_shift_range':0.2,
}

In [ ]:
Xy_train, Xy_val = utils.get_dataset_partitions_tf(procTrainImages, trainLabels, gen_kws=img_tweaks)


In [ ]:
model = ResNet(43)
model.build(input_shape = next(Xy_train)[0].shape)
model.layers[0].summary()

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) #early stopping to avoid overfitting and wasting time
filepath = 'best_resnet_weights.hdf5'
checkpoint =  tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')    #keeping saved weights of best result only
csv_logger = keras.callbacks.CSVLogger(f'log {dt.now().strftime("%Y-%m-%d %H:%M")} resnet.csv', append=True, separator=';')


model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(Xy_train, batch_size=32 ,epochs=30, validation_data=Xy_val, callbacks=[early_stop, checkpoint, csv_logger])


In [ ]:
model50 = Resnet50(43)
model50.build(input_shape = next(Xy_train)[0].shape)
model50.layers[0].summary()

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) #early stopping to avoid overfitting and wasting time
filepath = 'best_resnet34_weights.hdf5'
checkpoint =  tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')    #keeping saved weights of best result only
csv_logger = keras.callbacks.CSVLogger(f'log {dt.now().strftime("%Y-%m-%d %H:%M")} resnet50.csv', append=True, separator=';')


model50.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model50.fit(Xy_train, batch_size=32 ,epochs=30, validation_data=Xy_val, callbacks=[early_stop, checkpoint, csv_logger])
